In [105]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time


# Define your years, cause types, and measure values
years = ['2011', '2012']
#years = ['2011', '2012', '2013', '2014', '2015', '2016','2017' ,2018', '2019', '2020', '2021', '2022']
counties = ['Rural', 'Nonrural' ]
cause_types = ['Falls', 'All Other Unintentional Injury']

# Set up the WebDriver
s = Service("/Users/nikitajha/Documents/NikitaFinal/Coding Projects/Selenium/ChromeDriver/chromedriver")
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.default_directory": "/Users/nikitajha/Documents/NikitaFinal/Coding Projects/Bastos Lab/Source_3_Data_Folder",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
driver = webdriver.Chrome(service=s, options=chrome_options)

# Navigate to the webpage
driver.get("https://oasis.state.ga.us/oasis/webquery/qryER.aspx")
wait = WebDriverWait(driver, 30)

#Year dropdown 
def run_script_year(year):
  year_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "lstTime")))
  select_year = Select(year_dropdown)
  select_year.deselect_by_value("2022")
  select_year.select_by_value(year)

#County dropdown
def run_script_county(county):
  county_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "lstGeographies")))
  select_county = Select(county_dropdown)
  select_county.deselect_by_value("Georgia")
  select_county.select_by_value(county)

#Cause dropdown
def run_script_cause(cause):
  cause_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "drpCauseParent")))
  select_cause = Select(cause_dropdown)
  select_cause.select_by_value("External Causes")

  time.sleep(2)

  # Cause Sub-Type Dropdown
  cause_subtype_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "lstCauseChild")))
  select_cause_subtype = Select(cause_subtype_dropdown)
  select_cause_subtype.deselect_by_value("External Causes")
  select_cause_subtype.select_by_value(cause)
  
  #Click the submit button
  submit_button = wait.until(EC.element_to_be_clickable((By.ID, 'imgSubmit')))
  submit_button.click()

  # Wait for the export to Excel button to be clickable
  excel_button = wait.until(EC.element_to_be_clickable((By.ID, 'ImgSaveData')))
  excel_button.click()

  # Go back to home link and wait for the page to load
  driver.get("https://oasis.state.ga.us/oasis/webquery/qryER.aspx")
  wait.until(EC.presence_of_element_located((By.ID, 'lstTime')))
  wait.until(EC.presence_of_element_located((By.ID, "lstGeographies")))
  wait.until(EC.presence_of_element_located((By.ID, "drpCauseParent")))
  
def run_measure():
  wait = WebDriverWait(driver, 10)

  # Measure dropdown
  measure_dropdown = wait.until(EC.element_to_be_clickable((By.ID, "drpMeasure")))
  select = Select(measure_dropdown)
  select.deselect_by_value("Emergency Room Visits")
  select.select_by_value("Emergency Room Visit Rate")

  


for year in years:
  for county in counties:
    for cause in cause_types:
      try:
          run_measure()
          run_script_year(year)
          run_script_county(county)
          run_script_cause(cause)
      except Exception as e:
          print(f"An error occurred: {e}")
          try: 
            run_measure()
            run_script_year(year)
            run_script_county(county)
            run_script_cause(cause)
          except Exception as e:
            try: 
              print(f"An error 2 occurred: {e}")
              run_measure
              run_script_year(year)
              run_script_county(county)
              run_script_cause(cause)
            except Exception as e: 
              continue

driver.quit()





An error occurred: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x00000001033ee004 chromedriver + 4169732
1   chromedriver                        0x00000001033e5ff8 chromedriver + 4136952
2   chromedriver                        0x000000010303b500 chromedriver + 292096
3   chromedriver                        0x0000000103040808 chromedriver + 313352
4   chromedriver                        0x0000000103042780 chromedriver + 321408
5   chromedriver                        0x00000001030428ac chromedriver + 321708
6   chromedriver                        0x000000010307ca84 chromedriver + 559748
7   chromedriver                        0x0000000103075db4 chromedriver + 531892
8   chromedriver                        0x00000001030bb818 chromedr

In [104]:
import os
import sqlite3
import pandas as pd

def setup_database():
    # Database connection
    conn = sqlite3.connect('oasis2.db')
    c = conn.cursor()
    
    #Create table
    c.execute('''
        CREATE TABLE IF NOT EXISTS OasisData2 (
            County TEXT,
            Year INTEGER,
            Variable TEXT,
            Value REAL
        )
        ''')
    conn.commit()
    conn.close()
    

def process_and_insert_data(full_path, conn):
    # Read the first row, first cell to extract year, measure, and cause
    header_data = pd.read_excel(full_path, header=None, nrows=2)
    cause = (header_data.iloc[0, 0].split(",")[-1]).strip()
    year = str(header_data.iloc[1, 1])[0:4]
    variable = "ER Visit Rate-" + cause

    data = pd.read_excel(full_path, skiprows=2)
    #print("Column Names: ", data.columns)
    # Database cursor
    c = conn.cursor()

    for index, row in data.iterrows():
        county = str(row[' ']).strip()  # Assuming the ' ' column contains county names
        value = str(row['ER Visit Rate']).replace(",", "")  # Using the column name 'ER Visit Rate'
        if len(county.split(" ")) < 2 and county != "nan":
            c.execute('''
            INSERT INTO OasisData2 (County, Year, Variable, Value) 
            VALUES (?, ?, ?, ?)
            ''', (county, year, variable, value))

    conn.commit()

def export_to_csv():
    conn = sqlite3.connect("oasis2.db")
    query = """ 
    SELECT * FROM OasisData2 
    """
    
    df = pd.read_sql_query(query, conn)
    csv_file_path = "oasis_data2_temp.csv"
    df.to_csv(csv_file_path, index=False)
    conn.close()


if __name__ == "__main__":
    setup_database()
    folder_name = "Source_3_Data_Folder"
    conn = sqlite3.connect('oasis2.db')
    
    for filename in os.listdir(folder_name):
        if filename.endswith('.xlsx'):
            full_path = os.path.join(folder_name, filename)
            try:
                process_and_insert_data(full_path, conn)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
    
    export_to_csv()
    conn.close()
    os.remove('oasis2.db')